In [1]:
import requests

page = requests.get('http://www.sfgate.com/arrestrecords/san_francisco/')

print(page.status_code)

200


In [5]:
#%store -r arrests_df

arrests_df = pd.DataFrame()


In [6]:
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup(page.content, 'html.parser')

# find all links (each arrest on main page links to more detailed info about the arrest)
arrests_today = soup.find_all("a", class_="name")

arrest_report = []

# to grab elements from HTML table - input row and number of row element that we want
# i.e. 1 in the current arrest table and 0 in previous arrest table 
def grab_info(row, i):
    return row.find_all("td")[i].find(text=True)

for link in arrests_today:
    pg = requests.get(link.get("href"))
    sp = BeautifulSoup(pg.content, 'html.parser')
    
    # there are two tables we're interested in.. one with the current arrest info, one with previous arrests
    tables = sp.find_all("table", class_="table table-striped")
    # not every page has two tables though
    rows = tables[0].find_all("tr")
    name = grab_info(rows[0], 1)
    age = grab_info(rows[1], 1)
    loc = grab_info(rows[3], 1)
    date = grab_info(rows[4], 1)
    reason = grab_info(rows[6], 1)
    
    prev_arrests = []
    if (len(tables) == 2):
        for row in tables[1].find_all("tr"):
            prev_arrests.append(grab_info(row, 0))
        # remove 'Arrested For:' from the list
        prev_arrests.pop(0)
    
    num_prev_arrests = len(prev_arrests)
        
    arrest_report.append([name, age, loc, date, reason, prev_arrests, num_prev_arrests])
    
cols = ['name', 'age', 'city', 'date', 'reason', 'previous_arrests', 'num_prev_arrests']

# add today's arrests to a dataframe, starting with the correct index so we can append to the main df
todays_arrests = pd.DataFrame(arrest_report, columns=cols)
todays_arrests.index += len(arrests_df)

todays_arrests


,name,age,city,date,reason,previous_arrests,num_prev_arrests
0,Michael Kingsley,50,"San Francisco, CA",2067-10-27,Trespass By Driving On Private Property,"[Giving False Information To A Peace Officer, ...",2
1,Christopher Mar,24,"San Francisco, CA",2017-11-09,Petty Theft,"[Use/Under Influence Controlled Substance, Use...",15
2,Anthony Zapanta,34,"San Francisco, CA",2017-11-09,possession of drug paraphernalia,"[Receiving Stolen Property, Motor Vehicle, Pos...",8
3,James Simpson,34,"San Francisco, CA",2017-11-09,Possession of Controlled Substance,"[Warrants Or Holds Only, Possess Marijuana 28....",8
4,Scott Peron,21,"San Francisco, CA",2017-11-08,Adw Or Assault W/Force Likely To Produce Gbi,[],0
5,Belser Munozmorales,18,"San Francisco, CA",2017-11-08,Drive W/O License,[],0
6,Sandra Harris,55,"San Francisco, CA",2017-11-08,DUI Alcohol/Drugs,[],0
7,Edward Yanovsky,39,"San Francisco, CA",2017-11-08,Contempt Of Court:Disobey Court Order/Process,"[Resist/Obstruct/Delay Peace Officer, Contempt...",2
8,Charles Bursey,52,"San Francisco, CA",2017-11-08,Inflict Corporal Injury On Spouse/Cohabitant,[],0
9,Tashira Huntley,27,"San Francisco, CA",2017-11-08,Assault W/ Deadly Weapon Likely To Produce Gbi,"[Petty Theft, Conspiracy To Commit A Crime, Fr...",3


In [9]:
arrests_df = arrests_df.append(todays_arrests)
print(len(todays_arrests))
#%store arrests_df



25
